In [ ]:
import numpy as np
import re
import pandas as pd



In [ ]:
def preprocesses(texts):
  ret=[]
  for text in texts:
    text=text.lower()
    text=re.sub(r"[^a-zA-Z0-9]"," ",text)
    text=text.split(" ")
    while text.count(""):
      text.remove("")
    ret.append(text)
  return ret


In [ ]:
'''text1="The Premier League, also known exonymously as the English Premier League or the EPL (legal name: The Football Association Premier League Limited), is the top level of the English football league system. Contested by 20 clubs, it operates on a system of promotion and relegation with the English Football League (EFL). Seasons run from August to May with each team playing 38 matches (playing all 19 other teams both home and away). Most games are played on Saturday and Sunday afternoons."
text2="The Campeonato Nacional de Liga de Primera División, commonly known simply as Primera División in Spain, and as La Liga in English-speaking countries and officially as LaLiga Santander for sponsorship reasons, stylized as LaLiga, is the men's top professional football division of the Spanish football league system.Administered by the Liga Nacional de Fútbol Profesional, it is contested by 20 teams, with the three lowest-placed teams at the end of each season being relegated to the Segunda División and replaced by the top two teams and a play-off winner in that division."
text3="The Serie A, also called Serie A TIM for sponsorship reasons, is a professional league competition for football clubs located at the top of the Italian football league system and the winner is awarded the Scudetto and the Coppa Campioni d'Italia. It has been operating as a round-robin tournament for over ninety years since the 1929–30 season. It had been organized by the Direttorio Divisioni Superiori until 1943 and the Lega Calcio until 2010, when the Lega Serie A was created for the 2010–11 season. Serie A is regarded as one of the best football leagues in the world and it is often depicted as the most tactical and defensively sound national league.[3] Serie A was the world's strongest national league in 2020 according to IFFHS,[4] and is ranked third among European leagues according to UEFas league coefficient, behind La Liga and the Premier League and ahead of the Bundesliga and Ligue 1, which is based on the performance of Italian clubs in the Champions League and the Europa League during the previous five years. Serie A led the UEFA ranking from 1986 to 1988 and from 1990 to 1999."
'''
text1="the car is driven on the road"
text2 = "the truck is driven on the highway"
text3 = "the highway is for the truck"
texts=[text1,text2,text3]
text1,text2,text3=preprocesses(texts)

total=set(text1).union(set(text2)).union(set(text3))
A = dict.fromkeys(total, 0) 
B = dict.fromkeys(total, 0) 
C = dict.fromkeys(total, 0) 




In [ ]:
for i in text1:
  A[i]+=1
for i in text2:
  B[i]+=1
for i in text3:
  C[i]+=1

print(A,"\n",B,"\n",C)


{'driven': 1, 'is': 1, 'road': 1, 'for': 0, 'on': 1, 'the': 2, 'car': 1, 'highway': 0, 'truck': 0} 
 {'driven': 1, 'is': 1, 'road': 0, 'for': 0, 'on': 1, 'the': 2, 'car': 0, 'highway': 1, 'truck': 1} 
 {'driven': 0, 'is': 1, 'road': 0, 'for': 1, 'on': 0, 'the': 2, 'car': 0, 'highway': 1, 'truck': 1}


In [ ]:
def Tf(worddict,totalnum):
  tfdict={}
  for word,count in worddict.items():
     tfdict[word]=count/totalnum
  return tfdict

tfA=Tf(A,len(text1))
tfB=Tf(B,len(text2))
tfC=Tf(C,len(text3))

     

In [ ]:
def Idf(total,texts,N=2):
  
  idfdict=dict.fromkeys(total, 0) 
  for word in total:
    for text in texts:
      if word in text:
        idfdict[word]+=1
      
    

  for word,count in idfdict.items():
    if count !=0:
      idfdict[word]=np.log10(N/count)

  return idfdict

IDF=Idf(total,[text1,text2],2)
print(IDF)




{'driven': 0.0, 'is': 0.0, 'road': 0.3010299956639812, 'for': 0, 'on': 0.0, 'the': 0.0, 'car': 0.3010299956639812, 'highway': 0.3010299956639812, 'truck': 0.3010299956639812}


In [ ]:
def Tf_Idf(text,tf,idf):
  global total
  tf_idf_dict=dict.fromkeys(total,0)
  for i in text:
    tf_idf_dict[i]=tf[i]*idf[i]
  return tf_idf_dict

A_Tf_idf=Tf_Idf(text1,tfA,IDF)
B_Tf_idf=Tf_Idf(text2,tfB,IDF)




In [ ]:
df=pd.DataFrame([A_Tf_idf,B_Tf_idf])
df.head()


,driven,is,road,for,on,the,car,highway,truck
0,0.0,0.0,0.043004,0,0.0,0.0,0.043004,0.000000,0.000000
1,0.0,0.0,0.000000,0,0.0,0.0,0.000000,0.043004,0.043004


In [ ]:
z=df.loc[:, (df != 0).any(axis=0)]
z

,road,car,highway,truck
0,0.043004,0.043004,0.000000,0.000000
1,0.000000,0.000000,0.043004,0.043004


In [ ]:
C_Tf_idf=Tf_Idf(text3,tfC,IDF)
new=df.append(C_Tf_idf,ignore_index=True)

In [ ]:
new

,driven,is,road,for,on,the,car,highway,truck
0,0.0,0.0,0.043004,0.0,0.0,0.0,0.043004,0.000000,0.000000
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.043004,0.043004
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.050172,0.050172


In [ ]:
z=new.loc[:, (df != 0).any(axis=0)]
z

,road,car,highway,truck
0,0.043004,0.043004,0.000000,0.000000
1,0.000000,0.000000,0.043004,0.043004
2,0.000000,0.000000,0.050172,0.050172


In [ ]:
from scipy.spatial import distance

C_A=distance.euclidean(new.loc[new.index==2],new.loc[new.index==0])
C_B=distance.euclidean(new.loc[new.index==2],new.loc[new.index==1])

print(C_A,C_B)

if C_A>C_B:
   print("text2 is closer to text3")
else:
     print("text1 is closer to text3")


0.09345121294146519 0.010136207203550386
text2 is closer to text3
